## Parametric nonconvex optimization

Consider the following minimization problem ([reference](https://alphaville.github.io/optimization-engine/docs/example_rosenbrock_py))
$$
\begin{align}
    \operatorname*{Minimize}_{\|u\|\leq r}& \sum_{i=1}^{n_u - 1} b (u_{i+1} - u_{i}^2)^2 + (a-u_i)^2
    \\
    \text{subject to: }& 1.5 u_1 - u_2 = 0
    \\
    &u_3 - u_4 + 0.1 \leq 0
    \end{align}
$$
The parameter vector is $p=(a,b)$. 

Let us generate a parametric optimiser with $n_u=5$.

In [ ]:
import opengen as og
import casadi.casadi as cs

# Build parametric optimizer
# ------------------------------------
u = cs.SX.sym("u", 5)  # decision variables
p = cs.SX.sym("p", 2)  # parameters, p = (a, b)

# cost function:
phi = og.functions.rosenbrock(u, p)

# constraints:
c = cs.vertcat(1.5 * u[0] - u[1],
               cs.fmax(0.0, u[2] - u[3] + 0.1))

# simple bounds on decision variables:
bounds = og.constraints.Ball2(None, 1.5)

# problem formulation
problem = og.builder.Problem(u, p, phi) \
    .with_penalty_constraints(c)        \
    .with_constraints(bounds)

## Generate optimizer

The following code will generate your parametric optimizer. 

The auto-generated files will be stored in `optimizers/rosenbrock`.

In [ ]:
# Configure and build
# (This might take a while)
# ---------------------------------
build_config = og.config.BuildConfiguration()      \
    .with_build_directory("optimizers")            \
    .with_build_mode(og.config.BuildConfiguration.DEBUG_MODE)  \
    .with_tcp_interface_config()
meta = og.config.OptimizerMeta()                   \
    .with_optimizer_name("rosenbrock")
solver_config = og.config.SolverConfiguration()    \
    .with_tolerance(1e-5)                          \
    .with_delta_tolerance(1e-4)                    \
    .with_initial_penalty(1e3)                     \
    .with_penalty_weight_update_factor(5)
builder = og.builder.OpEnOptimizerBuilder(problem, meta,
                                          build_config, solver_config)
builder.build()

## Use the auto-generated optimizer

You may now use the auto-generated optimizer in Python.


<div class="alert alert-block alert-info">
<b>Tip:</b> First you need to <em>start</em> the server. Then, you can <em>call</em> it as many times as you like. When you're done, you can <em>kill</em> it.</div>

In [ ]:
# Start the optimisation server
# ----------------------------------
mng = og.tcp.OptimizerTcpManager('optimizers/rosenbrock')
mng.start()

In [ ]:
# Call the server
# ----------------------------------
response = mng.call([1.0, 50.0])
if response.is_ok():
    # Solver returned a solution
    solution_data = response.get()
    u_star = solution_data.solution
    exit_status = solution_data.exit_status
    solver_time = solution_data.solve_time_ms
    print("u_star = ", u_star)
    print("solved in = ", solver_time, "ms")
    

In [ ]:
# Kill the server
# ----------------------------------
mng.kill()

<div class="alert alert-block alert-info">
<b>Tip:</b> The auto-generated TCP server you built above listens for requests at port <code>8333</code>. Since it runs within this docker container, if you want to call it from another application outside this Jupyter notebook you need to forward the port - to do so, run the docker image with <code>-p 8333:8333</code>.</div>